The goal of this program is to take a list of search results from Archive of Our Own and return a more customized set of search results. In particular, this program returns the stories with a fic quality rating, simply calculated by dividing the number of kudos plus the number of bookmarks by the number of reads. Eventually, this calculation may become more nuanced, including accounting for series. 

Inputs:
    AO3 URL

In [1]:
from urllib import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests
import string
from lxml import html

base_url = "http://archiveofourown.org"
search_url = "http://archiveofourown.org/tags/Sherlock%20Holmes*s*Sarah%20Sawyer*s*John%20Watson/works?page="
#search_url = "http://archiveofourown.org/tags/Clint%20Barton*s*Phil%20Coulson/works?page="


In [2]:
def process_url(link_url):
    file = urlopen(link_url)
    soup = BeautifulSoup(file, "lxml")
    stories = soup.ol
    stories = soup.findAll("li", { "class" : "work blurb group" })
    return stories

Sample HTML from AO3 link


<li class="work blurb group" id="work_6092563" role="article">
<!--title, author, fandom-->
<div class="header module">
<h4 class="heading">
<a href="/works/6092563">Triadverse One-shots</a>
      by
        
      <!-- do not cache -->
<a href="/users/MelyndaR/pseuds/MelyndaR" rel="author">MelyndaR</a>
</h4>
<h5 class="fandoms heading">
<span class="landmark">Fandoms:</span>
<a class="tag" href="/tags/Sherlock%20(TV)/works">Sherlock (TV)</a>, <a class="tag" href="/tags/Sherlock%20Holmes%20*a*%20Related%20Fandoms/works">Sherlock Holmes &amp; Related Fandoms</a>, <a class="tag" href="/tags/Agents%20of%20S*d*H*d*I*d*E*d*L*d*D*d*%20(TV)/works">Agents of S.H.I.E.L.D. (TV)</a>, <a class="tag" href="/tags/Marvel%20Cinematic%20Universe/works">Marvel Cinematic Universe</a>, <a class="tag" href="/tags/Marvel/works">Marvel</a>
       
    </h5>
<!--required tags-->
<ul class="required-tags">
<li> <a aria-controls="#modal" class="help symbol question modal" href="/help/symbols-key.html" title="Symbols key"><span class="rating-general-audience rating" title="General Audiences"><span class="text">General Audiences</span></span></a></li>
<li> <a aria-controls="#modal" class="help symbol question modal" href="/help/symbols-key.html" title="Symbols key"><span class="warning-choosenotto warnings" title="Choose Not To Use Archive Warnings"><span class="text">Choose Not To Use Archive Warnings</span></span></a></li>
<li> <a aria-controls="#modal" class="help symbol question modal" href="/help/symbols-key.html" title="Symbols key"><span class="category-multi category" title="Multi"><span class="text">Multi</span></span></a></li>
<li> <a aria-controls="#modal" class="help symbol question modal" href="/help/symbols-key.html" title="Symbols key"><span class="complete-yes iswip" title="Complete Work"><span class="text">Complete Work</span></span></a></li>
</ul>
<p class="datetime">23 Feb 2016</p>
</div>
<!--warnings again, cast, freeform tags-->
<h6 class="landmark heading">Tags</h6>
<ul class="tags commas">
<li class="warnings"><strong><a class="tag" href="/tags/Choose%20Not%20To%20Use%20Archive%20Warnings/works">
Creator Chose Not To Use Archive Warnings</a></strong></li>
<li class="relationships">
<a class="tag" href="/tags/Irene%20Adler*s*Sherlock%20Holmes*s*Jim%20Moriarty/works">
Irene Adler/Sherlock Holmes/Jim Moriarty</a></li> 
<li class="relationships"><a class="tag" href="/tags/Anthea*s*Mycroft%20Holmes*s*Greg%20Lestrade/works">Anthea/Mycroft Holmes/Greg Lestrade</a></li> <li class="relationships"><a class="tag" href="/tags/Mycroft%20Holmes*s*Molly%20Hooper*s*Greg%20Lestrade/works">Mycroft Holmes/Molly Hooper/Greg Lestrade</a></li> <li class="relationships"><a class="tag" href="/tags/Sherlock%20Holmes*s*Sarah%20Sawyer*s*John%20Watson/works">Sherlock Holmes/Sarah Sawyer/John Watson</a></li> <li class="relationships"><a class="tag" href="/tags/Molly%20Hooper*s*Sebastian%20Moran*s*Jim%20Moriarty/works">Molly Hooper/Sebastian Moran/Jim Moriarty</a></li> <li class="relationships"><a class="tag" href="/tags/Sherlock%20Holmes*s*Mary%20Morstan*s*John%20Watson/works">Sherlock Holmes/Mary Morstan/John Watson</a></li> <li class="relationships"><a class="tag" href="/tags/Irene%20Adler*s*Sherlock%20Holmes*s*John%20Watson/works">Irene Adler/Sherlock Holmes/John Watson</a></li> <li class="relationships"><a class="tag" href="/tags/Sherlock%20Holmes*s*Molly%20Hooper*s*John%20Watson/works">Sherlock Holmes/Molly Hooper/John Watson</a></li> 
<li class="relationships"><a class="tag" href="/tags/Molly%20Hooper*s*Greg%20Lestrade*s*John%20Watson/works">Molly Hooper/Greg Lestrade/John Watson</a></li> <li class="relationships"><a class="tag" href="/tags/Irene%20Adler*s*Sherlock%20Holmes*s*Molly%20Hooper/works">Irene Adler/Sherlock Holmes/Molly Hooper</a></li>


<li class="characters"><a class="tag" href="/tags/Sherlock%20Holmes/works">Sherlock Holmes</a></li> <li class="characters"><a class="tag" href="/tags/Irene%20Adler/works">Irene Adler</a></li> <li class="characters"><a class="tag" href="/tags/Jim%20Moriarty/works">Jim Moriarty</a></li> <li class="characters"><a class="tag" href="/tags/Anthea%20(Sherlock)/works">Anthea (Sherlock)</a></li> <li class="characters"><a class="tag" href="/tags/Mycroft%20Holmes/works">Mycroft Holmes</a></li> <li class="characters"><a class="tag" href="/tags/Greg%20Lestrade/works">Greg Lestrade</a></li> <li class="characters"><a class="tag" href="/tags/Molly%20Hooper/works">Molly Hooper</a></li> <li class="characters"><a class="tag" href="/tags/Sarah%20Sawyer/works">Sarah Sawyer</a></li> <li class="characters"><a class="tag" href="/tags/John%20Watson/works">John Watson</a></li> <li class="characters"><a class="tag" href="/tags/Sebastian%20Moran/works">Sebastian Moran</a></li> <li class="characters"><a class="tag" href="/tags/Mary%20Morstan/works">Mary Morstan</a></li>
<li class="freeforms">
<a class="tag" href="/tags/Polyamory/works">Polyamory</a></li> <li class="freeforms">


<a class="tag" href="/tags/Triadverse/works">Triadverse</a></li>
</ul>
<!--summary-->
<h6 class="landmark heading">Summary</h6>
<blockquote class="userstuff summary">
<p>A collection of ten Sherlock triadverse one-shots, each being completely individual from the next.</p>
</blockquote>
<!--stats-->
<dl class="stats">
<dt class="language">Language:</dt>
<dd class="language">English</dd>
<dt class="words">Words:</dt>
<dd class="words">4,810</dd>
<dt class="chapters">Chapters:</dt>
<dd class="chapters">10/10</dd>
<dt class="comments">Comments:</dt>
<dd class="comments"><a href="/works/6092563?show_comments=true&amp;view_full_work=true#comments">13</a></dd>
<dt class="kudos">Kudos:</dt>
<dd class="kudos"><a href="/works/6092563?view_full_work=true#comments">17</a></dd>
<dt class="hits">Hits:</dt>
<dd class="hits">363</dd>
</dl>
</li>

In [3]:
def grab_max_page(link_url):
    file = urlopen(link_url)
    soup = BeautifulSoup(file, "lxml")
    pages = soup.find("ol", {"class": "pagination actions"})
    index = len(pages.contents) - 3
    max = pages.contents[index].a.contents[0]
    return max

In [4]:
def grab_story(story1):
    #Attempt to calculate score first
    
    try:   
        hits = float(story1.findAll("dd", { "class" : "hits" })[0].contents[0])
        kudos = float(story1.findAll("dd", {"class":"kudos"})[0].contents[0].contents[0])
        bookmarks = float(story1.findAll("dd", {"class": "bookmarks"})[0].contents[0].contents[0])
        score = (kudos + bookmarks)/hits * 100
    except IndexError:
        return
    else:  
        characters = []
        relationship = []
        header = story1.find("h4")
        story_url = base_url + header.a['href']
        title = header.a.contents[0]
        #Need to navigate eiher to 2nd link in header or to rel = "author"
        author = story1.find("a", {"rel": "author"}).contents[0]
        summary = story1.findAll("blockquote", {"class": "userstuff summary"})[0].contents[1].contents[0]

        #Store all tags as list
        tags = ""    
        language = ""
        word_count = ""
        chapters = ""
#        temp = story1.findAll("li", {"class": "characters"})[0]
#        for item in temp:
#            characters.append(item.contents[0].contents)
        date = story1.findAll("p", {"class":"datetime"})[0].contents[0]
        relationship = []
        temp = story1.findAll("li", {"class": "relationships"})
        for item in temp:
            relationship.append(item.contents[0].contents[0])        
    #    tags = story1.findAll
        summary = story1.findAll("blockquote", {"class": "userstuff summary"})[0].contents[1].contents[0]
        keys = ['Title','Author', 'Characters', 'Relationship', 'Score', 'Summary', 'Hits', 'Kudos', 'URL', 'Date', 'Bookmarks']    
        story_list.append(dict(zip(keys, [title, author, characters, relationship, score, summary, hits, kudos, story_url, date, bookmarks])))  


Look at: "<a rel="next" href="/tags/Sherlock%20Holmes*s*Sarah%20Sawyer*s*John%20Watson/works?page=2">
    Next &#8594;</a>"
    
<li class="next" title="next"><span class="disabled">Next &#8594;</span></li>

Look for bottom text. If it's present, then look no more for pages
    

In [12]:
story_list = []
page_number = 1
link_url = search_url + str(page_number)
print link_url
max = int(grab_max_page(link_url))
print "max = " + str(max)
print page_number
while page_number <= max:
    stories = process_url(link_url)
    for story in stories:
        grab_story(story)
    page_number += 1
    print page_number
    link_url = search_url + str(page_number)
    print link_url

http://archiveofourown.org/tags/Sherlock%20Holmes*s*Sarah%20Sawyer*s*John%20Watson/works?page=1
max = 2
1
2
http://archiveofourown.org/tags/Sherlock%20Holmes*s*Sarah%20Sawyer*s*John%20Watson/works?page=2
3
http://archiveofourown.org/tags/Sherlock%20Holmes*s*Sarah%20Sawyer*s*John%20Watson/works?page=3


In [13]:
story_list_df = pd.DataFrame(story_list)
sorted_stories = story_list_df.sort_values(by="Score", ascending=False, inplace=False, kind='quicksort', na_position='last')
print sorted_stories

             Author  Bookmarks Characters         Date     Hits  Kudos  \
21          Sadbhyl        8.0         []  10 Oct 2010    568.0   40.0   
9           Sadbhyl       19.0         []   1 Jul 2011   1525.0   83.0   
5      alizarin_nyc       11.0         []   4 Jan 2012   1893.0   92.0   
2       Mazarin221b       28.0         []  23 Aug 2012   3110.0  139.0   
11       Saathi1013       10.0         []   1 Apr 2011   2766.0  134.0   
10      Pyjamapants      150.0         []  29 May 2011  13900.0  461.0   
7    marysutherland        9.0         []   5 Nov 2011   1838.0   70.0   
1         lie_to_me        1.0         []   7 Sep 2013    877.0   29.0   
20       Saathi1013       24.0         []  13 Dec 2010   5319.0  151.0   
17       Saathi1013        2.0         []  28 Dec 2010   2629.0   81.0   
14       Saathi1013        2.0         []   1 Feb 2011   2797.0   85.0   
16       Saathi1013        2.0         []   4 Jan 2011   2523.0   74.0   
15       Saathi1013        2.0        